# Chapter 07: 학습 (Learning)

이 노트북에서는 Reflexion 패턴을 통한 에이전트 자기 개선을 학습합니다.

## 주요 내용
- Reflexion 패턴
- 실패 기반 개선
- 경험적 학습

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/TeeDDub/Building-Applications-with-AI-Agents/blob/main/notebook/ch07_learning.ipynb)


## 1. 패키지 설치


In [ ]:
!pip install -q langchain langchain-openai langgraph python-dotenv datasets torch transformers peft trl bitsandbytes


## 2. API 키 설정


In [ ]:
import os

try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    print("✅ Colab Secrets에서 API 키를 불러왔습니다.")
except:
    pass

if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = "sk-your-api-key-here"
    print("⚠️ API 키를 직접 입력해주세요.")


## 3. reflexion.py


Reflexion 패턴으로 실패 분석 및 개선 계획을 생성합니다.


In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, MessagesState, START
from langchain_core.messages import HumanMessage
import json

# 환경변수 확인
import os
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass  

if not os.getenv("OPENAI_API_KEY"):
    raise ValueError(
        "OPENAI_API_KEY가 설정되지 않았습니다."
        "환경변수 또는 .env 파일에서 설정해주세요."
    )

# LLM 초기화
llm = init_chat_model(model="gpt-5-mini", temperature=0)

reflections = []

def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    return {"messages": response}

reflexion_prompt = f"""어떤 환경에 놓인 뒤 특정 과제를 수행하라는 지시를 받았던 과거 경험의 기록이 주어질 것입니다.
당신은 그 과제를 완수하는 데 실패했습니다. 환경을 요약하려고 하지 말고, 과제를 완수하기 위해 시도했던 전략과 진행 경로에 대해 생각해 보십시오.
당신이 했어야 하지만 하지 못했던 구체적인 행동들을 참고하여, 그 실수를 보완할 수 있는 간결하고 새로운 실행 계획을 세우십시오.
예를 들어, A와 B는 시도했지만 C를 잊어버렸다면, 해당 환경에서 C를 달성하기 위해 어떤 행동을 했어야 하는지 환경에 특화된 행동들로 계획을 세우면 됩니다.
이 계획은 나중에 같은 과제를 다시 풀 때 필요하게 될 것입니다. "Plan"이라는 단어 뒤에 자신의 계획을 작성하십시오.

Instruction:
저는 유제품이 들어 있지 않고 사과 맛이 포함된 칩스 버라이어티 팩을 찾고 있으며, 가격은 30달러 미만이어야 합니다.

[Search]

Action: search[dairy free and apple variety pack of chips]
Observation:
[검색 결과로 돌아가기]
페이지 1 (총 결과: 50)
[다음 >]
[B07HRFJWP8]
Enjoy Life Foods Soft Baked Ovals, Breakfast Bars, Nut Free Bars, Soy Free, Dairy Free, Non GMO, Gluten Free, Vegan, Variety Pack, 4 Boxes (20 Bars Total)
$100.0
[B01KMHY5PG]
Enjoy Life Soft Baked Chewy Bars, Variety Pack, Nut Free Bars, Soy Free, Dairy Free, Gluten Free, 6 Boxes (30 Total Bars)
$21.49
[B008D2X8C4]
Enjoy Life Lentil Chips Variety Pack, Dairy Free Chips, Soy Free, Nut Free, Non GMO, Vegan, Gluten Free, 24 Bags (0.8 oz)
$100.0

Action: think[앞의 두 상품은 제가 원하는 것이 아닙니다. 세 번째 상품이 제가 찾는, 유제품이 들어 있지 않고 사과 맛이 포함된 칩스 버라이어티 팩입니다.]
Observation: 알겠습니다.

Action: click[B008D2X8C4]
Observation:
[검색 결과로 돌아가기]
[< 이전]
flavor name [dill and sour cream][garlic & parmesan][light sea salt][margherita pizza][thai chili lime][variety pack]
size [0.8 ounce (pack of 24)][4 ounce (pack of 12)]
가격(Price): $100.0
평점(Rating): N.A.
[Description]
[Features]
[Reviews]
[Buy Now]

Action: think[이 상품에는 'variety pack'과 '0.8 ounce (pack of 24)' 옵션이 있고, 구매하기에 좋아 보입니다.]
Observation: 알겠습니다.

Action: click[variety pack]
Observation: variety pack을 클릭하셨습니다.

Action: click[0.8 ounce (pack of 24)]
Observation: 0.8 ounce (pack of 24)를 클릭하셨습니다.

Action: click[Buy Now]

STATUS: FAIL

Plan:
"""


builder = StateGraph(MessagesState)
builder.add_node("reflexion", call_model)
builder.add_edge(START, "reflexion")
graph = builder.compile()

result = graph.invoke(
    {
        "messages": [
            HumanMessage(
                reflexion_prompt
            )
        ]
    }
)
reflections.append(result)

# 보기 좋게 출력
print("\n" + "="*80)
print("🔄 Reflexion 결과")
print("="*80 + "\n")

for i, msg in enumerate(result["messages"]):
    msg_type = msg.__class__.__name__
    
    if msg_type == "HumanMessage":
        print(f"입력 메시지:")
        print("-" * 80)
        # 프롬프트가 너무 길면 일부만 표시
        content = msg.content
        if len(content) > 500:
            print(content[:250] + "\n\n... (중략) ...\n\n" + content[-250:])
        else:
            print(content)
        print()
        
    elif msg_type == "AIMessage":
        print(f"AI 응답:")
        print("-" * 80)
        print(msg.content)
        print()

print("="*80)


## 4. experiential_learning.py


경험 기반 학습 흐름을 LangGraph로 구성합니다.


In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, MessagesState, START
from langchain_core.messages import HumanMessage

# 환경변수 확인
import os
try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass  

if not os.getenv("OPENAI_API_KEY"):
    raise ValueError(
        "OPENAI_API_KEY가 설정되지 않았습니다."
        "환경변수 또는 .env 파일에서 설정해주세요."
    )

# LLM 초기화
llm = init_chat_model(model="gpt-5-mini", temperature=0)

# LLM 호출 함수
def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    return {"messages": response}

class InsightAgent:
    def __init__(self):
        self.insights = []
        self.promoted_insights = []
        self.demoted_insights = []
        self.reflections = []

    def generate_insight(self, observation):
        # LLM을 사용하여 관찰에 기반한 인사이트를 생성합니다.
        messages = [HumanMessage(content=f"다음 관찰을 바탕으로 인사이트를 생성하세요: '{observation}'")]

        # 상태 그래프 생성
        builder = StateGraph(MessagesState)
        builder.add_node("generate_insight", call_model)
        builder.add_edge(START, "generate_insight")
        graph = builder.compile()

        # 메시지와 함께 그래프 호출
        result = graph.invoke({"messages": messages})

        # 생성된 인사이트 추출
        generated_insight = result["messages"][-1].content
        self.insights.append(generated_insight)
        print(f"생성된 인사이트: {generated_insight}")
        return generated_insight

    def promote_insight(self, insight):
        if insight in self.insights:
            self.insights.remove(insight)
            self.promoted_insights.append(insight)
            print(f"승격된 인사이트: {insight}")
        else:
            print(f"'{insight}'인사이트를 찾을 수 없습니다.")

    def demote_insight(self, insight):
        if insight in self.promoted_insights:
            self.promoted_insights.remove(insight)
            self.demoted_insights.append(insight)
            print(f"강등된 인사이트: {insight}")
        else:
            print(f"'{insight}'인사이트를 찾을 수 없습니다.")

    def edit_insight(self, old_insight, new_insight):
        # 모든 리스트에서 확인
        if old_insight in self.insights:
            index = self.insights.index(old_insight)
            self.insights[index] = new_insight
        elif old_insight in self.promoted_insights:
            index = self.promoted_insights.index(old_insight)
            self.promoted_insights[index] = new_insight
        elif old_insight in self.demoted_insights:
            index = self.demoted_insights.index(old_insight)
            self.demoted_insights[index] = new_insight
        else:
            print(f"'{old_insight}'인사이트를 찾을 수 없습니다.")
            return
        print(f"수정된 인사이트: '{old_insight}' -> '{new_insight}'")

    def show_insights(self):
        print("\n현재 인사이트:")
        print(f"인사이트: {self.insights}")
        print(f"승격된 인사이트: {self.promoted_insights}")
        print(f"강등된 인사이트: {self.demoted_insights}")

    def reflect(self, reflexion_prompt):
        # 성찰을 위한 상태 그래프 생성
        builder = StateGraph(MessagesState)
        builder.add_node("reflection", call_model)
        builder.add_edge(START, "reflection")
        graph = builder.compile()

        # 성찰 프롬프트와 함께 그래프 호출
        result = graph.invoke(
            {
                "messages": [
                    HumanMessage(
                        content=reflexion_prompt
                    )
                ]
            }
        )
        reflection = result["messages"][-1].content
        self.reflections.append(reflection)
        print(f"성찰: {reflection}")

agent = InsightAgent()

# 시뮬레이션된 관찰 시퀀스와 KPI 타겟 달성 여부
reports = [
    ("웹사이트 트래픽이 15% 증가했지만, 바운스율이 40%에서 55%로 급격히 증가했습니다.", 
        False),
    ("이메일 열람률이 25%로 향상되었지만, 20% 목표를 초과했습니다.", True),
    ("장바구니 포기율이 60%에서 68%로 증가했지만, 50% 목표를 놓쳤습니다.", 
        False),
    ("평균 주문 가치가 8% 증가했지만, 5% 증가 목표를 놓쳤습니다.", True),
    ("신규 구독자 수가 5% 감소했지만, 10% 성장 목표를 놓쳤습니다.", 
        False),
]
# 1) 보고서 기간 동안 인사이트 생성 및 우선순위 지정
for text, hit_target in reports:
    insight = agent.generate_insight(text)
    if hit_target:
        agent.promote_insight(insight)
    else:
        agent.demote_insight(insight)
# 2) 승격된 인사이트 중 하나를 사람이 참여하는 편집으로 개선
if agent.promoted_insights:
    original = agent.promoted_insights[0]
    agent.edit_insight(original, f'개선된 인사이트: {original} 방문자 경험 개선을 위한 랜딩 페이지 UX 변경 조사')
# 3) 에이전트의 최종 인사이트 상태 표시
agent.show_insights()
# 4) 최상위 인사이트를 바탕으로 개선 계획 성찰
reflection_prompt = (
    "승격된 인사이트를 바탕으로, 다음 분기에 실행할 수 있는 하나의 고영향 실험을 제안하세요:" + f"\n{agent.promoted_insights}"
)


## 5. supervised_fine_tuning.py


SFT 파이프라인을 구성해 모델을 미세 조정합니다.


In [ ]:
# fine_tune_function_calling.py
"""LoRA를 사용하여 함수 호출을 위한 LLM 파인튜닝을 수행하는 깔끔하고 모듈화된 스크립트입니다.

이 스크립트는 Bonus Unit 노트북의 단계들을 하나의 Python 진입점으로 통합합니다.
기본 사용법:

    HF_TOKEN=<your_token> python fine_tune_function_calling.py \
        --model google/gemma-2-2b-it \
        --dataset Jofthomas/hermes-function-calling-thinking-V1 \
        --output_dir gemma-2-2B-function-call-ft

모든 옵션을 보려면 `--help`를 참조하세요.
"""
from __future__ import annotations

import argparse
import os
from enum import Enum
from functools import partial
from pathlib import Path
from typing import Dict, List, Tuple

import torch
from datasets import DatasetDict, load_dataset
from peft import LoraConfig, PeftConfig, PeftModel, TaskType
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTConfig, SFTTrainer

###############################################################################
# 특수 토큰 및 채팅 템플릿 헬퍼
###############################################################################

class ChatmlSpecialTokens(str, Enum):
    tools = "<tools>"
    eotools = "</tools>"
    think = "<think>"
    eothink = "</think>"
    tool_call = "<tool_call>"
    eotool_call = "</tool_call>"
    tool_response = "<tool_response>"
    eotool_response = "</tool_response>"
    pad_token = "<pad>"
    eos_token = "<eos>"

    @classmethod
    def list(cls) -> List[str]:
        return [c.value for c in cls]

CHAT_TEMPLATE = (
    "{{ bos_token }}"
    "{% if messages[0]['role'] == 'system' %}{{ raise_exception('System role not supported') }}{% endif %}"
    "{% for message in messages %}"
    "{{ '<start_of_turn>' + message['role'] + '\n' + message['content']|trim + '<end_of_turn><eos>\n' }}"
    "{% endfor %}"
    "{% if add_generation_prompt %}{{'<start_of_turn>model\n'}}{% endif %}"
)

###############################################################################
# 데이터셋 전처리
###############################################################################

def _merge_system_into_first_user(messages: List[Dict[str, str]]) -> None:
    """선행 시스템 메시지를 후속 사용자 메시지에 병합합니다."""
    if messages and messages[0]["role"] == "system":
        system_content = messages[0]["content"]
        messages.pop(0)
        if not messages or messages[0]["role"] != "human":
            raise ValueError("시스템 메시지 다음에 사용자 메시지가 있어야 합니다.")
        messages[0][
            "content"
        ] = (
            f"{system_content}또한, 함수를 호출하기 전에 시간을 갖고 "
            "호출할 함수를 계획하세요. 생각하는 과정을 "
            "<think>{your thoughts}</think> 사이에 작성하세요.\n\n" + messages[0]["content"]
        )


def build_preprocess_fn(tokenizer):
    """원시 샘플을 토크나이즈된 프롬프트로 매핑하는 함수를 반환합니다."""
    def _preprocess(sample):
        messages = sample["messages"].copy()
        _merge_system_into_first_user(messages)
        prompt = tokenizer.apply_chat_template(messages, tokenize=False)
        return {"text": prompt}

    return _preprocess


def load_and_prepare_dataset(ds_name: str, tokenizer, max_train: int, max_eval: int) -> DatasetDict:
    """데이터셋을 로드하고 전처리 및 학습/테스트 분할을 적용합니다."""
    raw = load_dataset(ds_name).rename_column("conversations", "messages")
    processed = raw.map(build_preprocess_fn(tokenizer), remove_columns="messages")
    split = processed["train"].train_test_split(test_size=0.1, seed=42)
    split["train"] = split["train"].select(range(max_train))
    split["test"] = split["test"].select(range(max_eval))
    return split

###############################################################################
# 모델 및 토크나이저 헬퍼
###############################################################################

def build_tokenizer(model_name: str):
    tokenizer = AutoTokenizer.from_pretrained(
        model_name,
        pad_token=ChatmlSpecialTokens.pad_token.value,
        additional_special_tokens=ChatmlSpecialTokens.list(),
    )
    tokenizer.chat_template = CHAT_TEMPLATE
    return tokenizer


def build_model(model_name: str, tokenizer, load_4bit: bool = False):
    kwargs = {
        "attn_implementation": "eager",
        "device_map": "auto",
    }
    kwargs["quantization_config"] = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )
    model = AutoModelForCausalLM.from_pretrained(model_name, **kwargs)
    model.resize_token_embeddings(len(tokenizer))
    return model

###############################################################################
# PEFT / LoRA 헬퍼
###############################################################################

def build_lora_config(r: int = 16, alpha: int = 64, dropout: float = 0.05) -> LoraConfig:
    return LoraConfig(
        r=r,
        lora_alpha=alpha,
        lora_dropout=dropout,
        target_modules=[
            "gate_proj",
            "q_proj",
            "k_proj",
            "v_proj",
            "o_proj",
            "up_proj",
            "down_proj",
            "lm_head",
            "embed_tokens",
        ],
        task_type=TaskType.CAUSAL_LM,
    )

###############################################################################
# 학습
###############################################################################

def train(
    model,
    tokenizer,
    dataset: DatasetDict,
    peft_cfg: LoraConfig,
    output_dir: str,
    epochs: int = 1,
    lr: float = 1e-4,
    batch_size: int = 1,
    grad_accum: int = 4,
    max_seq_len: int = 1500,
):
    train_args = SFTConfig(
        output_dir=output_dir,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        gradient_accumulation_steps=grad_accum,
        save_strategy="no",
        eval_strategy="epoch",
        logging_steps=5,
        learning_rate=lr,
        num_train_epochs=epochs,
        max_grad_norm=1.0,
        warmup_ratio=0.1,
        lr_scheduler_type="cosine",
        report_to=None,
        bf16=True,
        gradient_checkpointing=True,
        gradient_checkpointing_kwargs={"use_reentrant": False},
        packing=True,
    )

    trainer = SFTTrainer(
        model=model,
        args=train_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        processing_class=tokenizer,
        peft_config=peft_cfg,
    )

    trainer.train()
    trainer.save_model()
    return trainer

###############################################################################
# 명령줄 인터페이스(CLI)
###############################################################################

def parse_args() -> argparse.Namespace:
    parser = argparse.ArgumentParser(description="LoRA를 사용하여 함수 호출을 위한 LLM 파인튜닝을 수행합니다.")
    parser.add_argument("--model", default="microsoft/Phi-3-mini-4k-instruct", help="기본 모델 이름 또는 경로")
    parser.add_argument("--dataset", default="Jofthomas/hermes-function-calling-thinking-V1", help="HuggingFace 데이터셋")
    parser.add_argument("--output_dir", default="ch07/fine_tuned_model/gemma-2-2B-function-call-ft", help="체크포인트 저장 경로")
    parser.add_argument("--epochs", type=int, default=1)
    parser.add_argument("--batch_size", type=int, default=1)
    parser.add_argument("--grad_accum", type=int, default=4)
    parser.add_argument("--lr", type=float, default=1e-4)
    parser.add_argument("--max_train", type=int, default=100, help="빠른 실행을 위한 학습 데이터 행 수")
    parser.add_argument("--max_eval", type=int, default=10, help="빠른 실행을 위한 평가 데이터 행 수")
    parser.add_argument("--push_to_hub", action="store_true")
    parser.add_argument("--hf_username", default=None, help="모델 푸시를 위한 HuggingFace 사용자명")
    parser.add_argument("--load_4bit", action="store_true", help="4비트 양자화 모드로 기본 모델 로드")
    return parser.parse_args()


def maybe_push_to_hub(trainer: SFTTrainer, tokenizer, username: str, output_dir: str):
    if not username:
        print("HuggingFace 사용자명이 제공되지 않아 push_to_hub을 건너뜁니다.")
        return
    repo = f"{username}/{Path(output_dir).name}"
    print(f"\n어댑터 및 토크나이저를 https://huggingface.co/{repo} 에 푸시하는 중 …")
    trainer.push_to_hub(repo)
    tokenizer.push_to_hub(repo, token=os.environ.get("HF_TOKEN"))

###############################################################################
# 진입점
###############################################################################

def main():
    args = parse_args()

    tokenizer = build_tokenizer(args.model)
    model = build_model(args.model, tokenizer, load_4bit=args.load_4bit)

    dataset = load_and_prepare_dataset(
        args.dataset, tokenizer, max_train=args.max_train, max_eval=args.max_eval
    )

    lora_cfg = build_lora_config()
    results = train(
        model,
        tokenizer,
        dataset,
        lora_cfg,
        output_dir=args.output_dir,
        epochs=args.epochs,
        lr=args.lr,
        batch_size=args.batch_size,
        grad_accum=args.grad_accum,
    )

    print("\n학습 완료! 🎉")
    print(results)


if __name__ == "__main__":
    main()


## 6. direct_preference_optimization.py


DPO로 선호도 기반 학습을 수행합니다.


In [ ]:
# fine_tune_helpdesk_dpo.py
# DPO(Direct Preference Optimization)를 사용한 헬프데스크 모델 파인튜닝
import torch, os
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from trl import DPOConfig, DPOTrainer
import logging

BASE_SFT_CKPT = "microsoft/Phi-3-mini-4k-instruct"
DPO_DATA      = "ch07/training_data/dpo_it_help_desk_training_data.jsonl"                   # -> 경로 또는 HF 데이터셋
OUTPUT_DIR    = "ch07/fine_tuned_model/phi3-mini-helpdesk-dpo"

# 1️⃣ 모델 + 토크나이저 로드
tok = AutoTokenizer.from_pretrained(BASE_SFT_CKPT, padding_side="right",
                                    trust_remote_code=True)

logger = logging.getLogger(__name__)
if not os.path.exists(BASE_SFT_CKPT):
    logger.warning("로컬 경로를 찾을 수 없습니다. Hub에서 '%s'를 다운로드합니다.", BASE_SFT_CKPT)

# 2️⃣ 양자화 설정 (4비트)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,               # 가중치를 4비트로 양자화
    bnb_4bit_use_double_quant=True,  # 선택사항: 중첩 양자화
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 3️⃣ 기본 모델 로드 (양자화 적용)
base = AutoModelForCausalLM.from_pretrained(
    BASE_SFT_CKPT,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)

# 4️⃣ LoRA 설정 및 모델 준비
lora_cfg = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
   target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj",
                    "up_proj", "down_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(base, lora_cfg)
print("✅  Phi-3 모델 로드 완료:", model.config.hidden_size, "hidden dim")

# 5️⃣ 데이터셋 로드
ds = load_dataset("json", data_files=DPO_DATA, split="train")

# 6️⃣ 학습 인자 설정 (사용하지 않음, DPOConfig로 대체)
train_args = TrainingArguments(
    output_dir      = OUTPUT_DIR,
    per_device_train_batch_size = 4,
    gradient_accumulation_steps = 4,
    learning_rate   = 5e-6,
    num_train_epochs= 3,
    logging_steps   = 10,
    save_strategy   = "epoch",
    bf16            = True,
    report_to       = None,
)

# 7️⃣ DPO 학습 설정
dpo_args = DPOConfig(
    output_dir              = "phi3-mini-helpdesk-dpo",
    per_device_train_batch_size  = 4,
    gradient_accumulation_steps  = 4,
    learning_rate           = 5e-6,
    num_train_epochs        = 3.0,
    bf16                    = True,
    logging_steps           = 10,
    save_strategy           = "epoch",
    report_to               = None,
    beta                    = 0.1,
    loss_type               = "sigmoid",
    label_smoothing         = 0.0,
    max_prompt_length       = 4096,
    max_completion_length   = 4096,
    max_length              = 8192,
    padding_value           = tok.pad_token_id,
    label_pad_token_id      = tok.pad_token_id,
    truncation_mode         = "keep_end",
    generate_during_eval    = False,
    disable_dropout         = False,
    reference_free          = True,
    model_init_kwargs       = None,
    ref_model_init_kwargs   = None,
)

# 8️⃣ DPO 트레이너 초기화
trainer = DPOTrainer(
    model,
    ref_model=None,           # reference_free=True이므로 참조 모델 불필요
    args=dpo_args,
    train_dataset=ds,
)

# 9️⃣ 학습 실행 및 모델 저장
trainer.train()
trainer.save_model()
tok.save_pretrained(OUTPUT_DIR)
print(f"✅  모델 저장 완료: {OUTPUT_DIR}")


## 7. reinforcement_learning_with_verifiable_rewards.py


검증 가능한 보상 기반 RL 실험을 구성합니다.


In [ ]:
from datasets import load_dataset
from trl import GRPOTrainer, GRPOConfig
import torch
import re
import json
from typing import List, Dict, Any

# RLVR(Reinforcement Learning with Verifiable Rewards) 훈련 데이터 경로
DPO_DATA = "ch07/training_data/rlvr_it_help_desk_training_data.jsonl"

# 데이터셋 로드
dataset = load_dataset("json", data_files=DPO_DATA, split="train")

def reward_tool_call_quality(completions: List[str], **kwargs) -> List[float]:
    """
    함수 호출 품질에 대한 세밀한 보상 함수.

    보상 체계:
    - 올바른 도구명 + 유효한 JSON + 필수 매개변수: +1.0
    - 올바른 도구명 + 유효한 JSON + 누락된 매개변수: +0.5
    - 올바른 도구명 + 잘못된 JSON: +0.2
    - 잘못된 도구명 + 유효한 JSON: -0.3
    - 도구 호출 없음 또는 완전히 잘못됨: -1.0
    """
    labels = kwargs.get('label', [])
    expected_params = kwargs.get('required_params', [])  # 선택사항: 필수 매개변수 이름 목록

    rewards = []
    num_generations = kwargs.get('num_generations', getattr(trainer.args, 'num_generations', 1))

    for i, completion in enumerate(completions):
        # 이 완성 결과에 대한 레이블 가져오기
        label_idx = i // num_generations
        if label_idx >= len(labels):
            rewards.append(-1.0)
            continue

        label = labels[label_idx]
        expected_tool = label.lower().strip()

        # 완성 결과에서 도구 호출 추출
        tool_match = re.search(
            r'<tool_call>\s*(\{.*?\})\s*</tool_call>',
            completion,
            re.DOTALL
        )

        if not tool_match:
            # 도구 호출을 찾지 못함
            rewards.append(-1.0)
            continue

        tool_json_str = tool_match.group(1)

        # JSON 파싱 시도
        try:
            tool_call = json.loads(tool_json_str)
        except json.JSONDecodeError:
            # 잘못된 JSON - 하지만 최소한 도구 호출을 시도함
            # 형식이 잘못된 문자열에 도구명이 나타나는지 확인
            if expected_tool in tool_json_str.lower():
                rewards.append(0.2)
            else:
                rewards.append(-0.5)
            continue

        # 유효한 JSON - 이제 도구명 확인
        tool_name = tool_call.get('name', '').lower().strip()

        # 도구명이 올바른지 확인
        tool_name_correct = (
            expected_tool in tool_name or
            tool_name in expected_tool or
            tool_name == expected_tool
        )

        if not tool_name_correct:
            # 잘못된 도구이지만 유효한 JSON
            rewards.append(-0.3)
            continue

        # 올바른 도구명 + 유효한 JSON
        # 제공된 경우 매개변수 확인
        if expected_params and label_idx < len(expected_params):
            required = expected_params[label_idx]
            provided_params = tool_call.get('parameters', tool_call.get('arguments', {}))

            if isinstance(provided_params, dict):
                has_all_required = all(
                    param in provided_params and provided_params[param] not in [None, '', []]
                    for param in required
                )

                if has_all_required:
                    rewards.append(1.0)  # 완벽!
                else:
                    rewards.append(0.5)  # 올바른 도구이지만 매개변수 누락
            else:
                rewards.append(0.5)  # 잘못된 형식의 매개변수
        else:
            # 매개변수 검사 없음 - 도구명만 검증
            rewards.append(1.0)

    return rewards


def reward_format_compliance(completions: List[str], **kwargs) -> List[float]:
    """
    형식 준수를 위한 보상 함수.
    올바른 XML 태그, JSON 구조 등을 확인합니다.
    """
    rewards = []

    for completion in completions:
        reward = 0.0

        # 올바른 tool_call 태그 확인
        if '<tool_call>' in completion and '</tool_call>' in completion:
            reward += 0.3

        # 균형 잡힌 중괄호 확인
        if completion.count('{') == completion.count('}'):
            reward += 0.2

        # 키 주위의 따옴표 확인 (기본 JSON 형식)
        tool_match = re.search(r'<tool_call>(.*?)</tool_call>', completion, re.DOTALL)
        if tool_match:
            tool_content = tool_match.group(1).strip()
            # 기본 확인: "name"이 따옴표와 함께 있어야 함
            if '"name"' in tool_content or "'name'" in tool_content:
                reward += 0.3

            # 파싱 가능한지 확인
            try:
                json.loads(tool_content)
                reward += 0.2  # 유효한 JSON에 대한 보너스
            except:
                pass

        rewards.append(reward)

    return rewards


# 가중치를 사용하여 여러 보상 함수 결합
def combined_reward(completions: List[str], **kwargs) -> List[float]:
    """여러 보상 신호의 가중치 결합."""
    quality_rewards = reward_tool_call_quality(completions, **kwargs)
    format_rewards = reward_format_compliance(completions, **kwargs)

    # 형식보다 품질에 더 높은 가중치 부여
    combined = [
        0.8 * q + 0.2 * f
        for q, f in zip(quality_rewards, format_rewards)
    ]

    return combined


# GRPO(Group Relative Policy Optimization) 설정
grpo_config = GRPOConfig(
    output_dir="ch07/fine_tuned_model/qwen-helpdesk-rlvr",
    num_generations=4,  # 프롬프트당 여러 완성 결과 생성
    learning_rate=5e-6,
    per_device_train_batch_size=4,  # num_generations로 나누어떨어져야 함
    num_train_epochs=3,
    logging_steps=1,
    save_strategy="epoch",
    report_to=None,
)

# GRPO(Group Relative Policy Optimization) 트레이너 설정
trainer = GRPOTrainer(
    model="Qwen/Qwen2-0.5B-Instruct",
    reward_funcs=combined_reward,
    train_dataset=dataset,
    args=grpo_config,
)

print(f"✅ 모델 로드 완료")
print(f"훈련 장치: {trainer.model.device}")

# 훈련 시작
print("훈련을 시작합니다...")
trainer.train()

print(f"✅ 훈련 완료! 모델 저장: {grpo_config.output_dir}")
